<a href="https://colab.research.google.com/github/murmurmaomao/-Data-Science-for-Smart-Environments_GRS35306/blob/main/Hyperparameter_Optimization_LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Set the lightgbm GPU environment

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM

Cloning into 'LightGBM'...
remote: Enumerating objects: 30949, done.
remote: Counting objects: 100% (5441/5441), done.
remote: Compressing objects: 100% (474/474), done.
remote: Total 30949 (delta 5192), reused 5028 (delta 4965), pack-reused 25508
Receiving objects: 100% (30949/30949), 21.16 MiB | 16.39 MiB/s, done.
Resolving deltas: 100% (23116/23116), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21733, done.        
remote: Counting objects: 100% (5/5), done.        
remot

In [ ]:
!apt-get install -y -qq libboost-all-dev

In [ ]:
%%bash
cd LightGBM
rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Looking for CL_VERSION_3_0
-- Looking for CL_VERSION_3_0 - found
-- Found OpenCL: /usr/local/cuda/lib64/libOpenCL.so (found version "3.0") 
-- OpenCL include directory: /usr/local/cuda/include
-- Found Boost: /usr/lib/x86_64-linux-gnu/cmake/Boost-1.74.0/BoostConfig.cmake (found suitable version "1.74.0", minimum required is "1.56.0") found 

rm: cannot remove 'build': No such file or directory
In file included from /content/LightGBM/external_libs/compute/include/boost/compute/cl.hpp:19,
                 from /content/LightGBM/external_libs/compute/include/boost/compute/config.hpp:16,
                 from /content/LightGBM/external_libs/compute/include/boost/compute/buffer.hpp:14,
                 from /content/LightGBM/external_libs/compute/include/boost/compute/core.hpp:18,
                 from /content/LightGBM/src/treelearner/gpu_tree_learner.h:33,
                 from /content/LightGBM/src/treelearner/gpu_tree_learner.cpp:7:
/usr/local/cuda/include/CL/cl.h:26:104: note: ‘#pragma message: cl_version.h: CL_TARGET_OPENCL_VERSION is not defined. Defaulting to 300 (OpenCL 3.0)’
   26 | #pragma message("cl_version.h: CL_TARGET_OPENCL_VERSION is not defined. Defaulting to 300 (OpenCL 3.0)")
      |                                                                                                        ^
In file included from

In [ ]:
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!rm -r LightGBM

# Import pakeages

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import os
import gc
import datetime

import numpy as np
import pandas as pd
import polars as pl
import plotly.express as px
import pyarrow as pa

import lightgbm as lgb
from lightgbm import LGBMRegressor
import optuna
from sklearn.metrics import mean_absolute_error

import holidays

# Class

## Load data

In [ ]:
class DataStorage:
    # root = "..\data"
    root = "/content/drive/MyDrive/predict-energy-behavior-of-prosumers/Data"
    # root = "/kaggle/input/predict-energy-behavior-of-prosumers"

    data_cols = [
        "target",
        "county",
        "is_business",
        "product_type",
        "is_consumption",
        "datetime",
        "row_id",
    ]
    client_cols = [
        "product_type",
        "county",
        "eic_count",
        "installed_capacity",
        "is_business",
        "date",
    ]
    gas_prices_cols = ["forecast_date", "lowest_price_per_mwh", "highest_price_per_mwh"]
    electricity_prices_cols = ["forecast_date", "euros_per_mwh"]
    forecast_weather_cols = [
        "latitude",
        "longitude",
        "origin_datetime",
        "hours_ahead",
        "temperature",
        "dewpoint",
        "cloudcover_high",
        "cloudcover_low",
        "cloudcover_mid",
        "cloudcover_total",
        "10_metre_u_wind_component",
        "10_metre_v_wind_component",
        "forecast_datetime",
        "direct_solar_radiation",
        "surface_solar_radiation_downwards",
        "snowfall",
        "total_precipitation",
    ]
    historical_weather_cols = [
        "datetime",
        "temperature",
        "dewpoint",
        "rain",
        "snowfall",
        "surface_pressure",
        "cloudcover_total",
        "cloudcover_low",
        "cloudcover_mid",
        "cloudcover_high",
        "windspeed_10m",
        "winddirection_10m",
        "shortwave_radiation",
        "direct_solar_radiation",
        "diffuse_radiation",
        "latitude",
        "longitude",
    ]
    location_cols = ["longitude", "latitude", "county"]
    target_cols = [
        "target",
        "county",
        "is_business",
        "product_type",
        "is_consumption",
        "datetime",
    ]

    def __init__(self):
        self.df_data = pl.read_csv(
            os.path.join(self.root, "train.csv"),
            columns=self.data_cols,
            try_parse_dates=True,
        )
        self.df_client = pl.read_csv(
            os.path.join(self.root, "client.csv"),
            columns=self.client_cols,
            try_parse_dates=True,
        )
        self.df_gas_prices = pl.read_csv(
            os.path.join(self.root, "gas_prices.csv"),
            columns=self.gas_prices_cols,
            try_parse_dates=True,
        )
        self.df_electricity_prices = pl.read_csv(
            os.path.join(self.root, "electricity_prices.csv"),
            columns=self.electricity_prices_cols,
            try_parse_dates=True,
        )
        self.df_forecast_weather = pl.read_csv(
            os.path.join(self.root, "forecast_weather.csv"),
            columns=self.forecast_weather_cols,
            try_parse_dates=True,
        )
        self.df_historical_weather = pl.read_csv(
            os.path.join(self.root, "historical_weather.csv"),
            columns=self.historical_weather_cols,
            try_parse_dates=True,
        )
        self.df_weather_station_to_county_mapping = pl.read_csv(
            os.path.join(self.root, "weather_station_to_county_mapping.csv"),
            columns=self.location_cols,
            try_parse_dates=True,
        )
        self.df_data = self.df_data.filter(
            (pl.col("datetime") >= pd.to_datetime("2022-01-01"))&
            (pl.col("datetime") <= pd.to_datetime("2023-05-17"))
        )
        self.df_target = self.df_data.select(self.target_cols)

        self.schema_data = self.df_data.schema
        self.schema_client = self.df_client.schema
        self.schema_gas_prices = self.df_gas_prices.schema
        self.schema_electricity_prices = self.df_electricity_prices.schema
        self.schema_forecast_weather = self.df_forecast_weather.schema
        self.schema_historical_weather = self.df_historical_weather.schema
        self.schema_target = self.df_target.schema

        self.df_weather_station_to_county_mapping = (
            self.df_weather_station_to_county_mapping.with_columns(
                pl.col("latitude").cast(pl.datatypes.Float32),
                pl.col("longitude").cast(pl.datatypes.Float32),
            )
        )

    def update_with_new_data(
        self,
        df_new_client,
        df_new_gas_prices,
        df_new_electricity_prices,
        df_new_forecast_weather,
        df_new_historical_weather,
        df_new_target,
    ):
        df_new_client = pl.from_pandas(
            df_new_client[self.client_cols], schema_overrides=self.schema_client
        )
        df_new_gas_prices = pl.from_pandas(
            df_new_gas_prices[self.gas_prices_cols],
            schema_overrides=self.schema_gas_prices,
        )
        df_new_electricity_prices = pl.from_pandas(
            df_new_electricity_prices[self.electricity_prices_cols],
            schema_overrides=self.schema_electricity_prices,
        )
        df_new_forecast_weather = pl.from_pandas(
            df_new_forecast_weather[self.forecast_weather_cols],
            schema_overrides=self.schema_forecast_weather,
        )
        df_new_historical_weather = pl.from_pandas(
            df_new_historical_weather[self.historical_weather_cols],
            schema_overrides=self.schema_historical_weather,
        )
        df_new_target = pl.from_pandas(
            df_new_target[self.target_cols], schema_overrides=self.schema_target
        )

        self.df_client = pl.concat([self.df_client, df_new_client]).unique(
            ["date", "county", "is_business", "product_type"]
        )
        self.df_gas_prices = pl.concat([self.df_gas_prices, df_new_gas_prices]).unique(
            ["forecast_date"]
        )
        self.df_electricity_prices = pl.concat(
            [self.df_electricity_prices, df_new_electricity_prices]
        ).unique(["forecast_date"])
        self.df_forecast_weather = pl.concat(
            [self.df_forecast_weather, df_new_forecast_weather]
        ).unique(["forecast_datetime", "latitude", "longitude", "hours_ahead"])
        self.df_historical_weather = pl.concat(
            [self.df_historical_weather, df_new_historical_weather]
        ).unique(["datetime", "latitude", "longitude"])
        self.df_target = pl.concat([self.df_target, df_new_target]).unique(
            ["datetime", "county", "is_business", "product_type", "is_consumption"]
        )



## Feature Enegineering

In [ ]:
class FeaturesGenerator:
    def __init__(self, data_storage):
        self.data_storage = data_storage
        self.estonian_holidays = list(
            holidays.country_holidays("EE", years=range(2021, 2026)).keys()
        )

    def _add_general_features(self, df_features):
        df_features = (
            df_features.with_columns(
                pl.col("datetime").dt.ordinal_day().alias("dayofyear"),
                pl.col("datetime").dt.hour().alias("hour"),
                pl.col("datetime").dt.day().alias("day"),
                pl.col("datetime").dt.weekday().alias("weekday"),
                pl.col("datetime").dt.month().alias("month"),
                pl.col("datetime").dt.year().alias("year"),
            )
            .with_columns(
                pl.concat_str(
                    "county",
                    "is_business",
                    "product_type",
                    "is_consumption",
                    separator="_",
                ).alias("segment"),
            )
            .with_columns(
                (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"),
                (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
                (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
                (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
            )
        )
        return df_features

    def _add_client_features(self, df_features):
        df_client = self.data_storage.df_client

        df_features = df_features.join(
            df_client.with_columns(
                (pl.col("date") + pl.duration(days=2)).cast(pl.Date)
            ),
            on=["county", "is_business", "product_type", "date"],
            how="left",
        )
        return df_features

    def is_country_holiday(self, row):
        return (
            datetime.date(row["year"], row["month"], row["day"])
            in self.estonian_holidays
        )

    def _add_holidays_features(self, df_features):
        df_features = df_features.with_columns(
            pl.struct(["year", "month", "day"])
            .apply(self.is_country_holiday)
            .alias("is_country_holiday")
        )
        return df_features

    def _add_forecast_weather_features(self, df_features):
        df_forecast_weather = self.data_storage.df_forecast_weather
        df_weather_station_to_county_mapping = (
            self.data_storage.df_weather_station_to_county_mapping
        )

        df_forecast_weather = (
            df_forecast_weather.rename({"forecast_datetime": "datetime"})
            .filter((pl.col("hours_ahead") >= 22) & pl.col("hours_ahead") <= 45)
            .drop("hours_ahead")
            .with_columns(
                pl.col("latitude").cast(pl.datatypes.Float32),
                pl.col("longitude").cast(pl.datatypes.Float32),
            )
            .join(
                df_weather_station_to_county_mapping,
                how="left",
                on=["longitude", "latitude"],
            )
            .drop("longitude", "latitude", "origin_datetime")
        )

        df_forecast_weather_date = (
            df_forecast_weather.group_by("datetime").mean().drop("county")
        )

        df_forecast_weather_local = (
            df_forecast_weather.filter(pl.col("county").is_not_null())
            .group_by("county", "datetime")
            .mean()
        )

        for hours_lag in [0, 7 * 24]:
            df_features = df_features.join(
                df_forecast_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on="datetime",
                how="left",
                suffix=f"_forecast_{hours_lag}h",
            )
            df_features = df_features.join(
                df_forecast_weather_local.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on=["county", "datetime"],
                how="left",
                suffix=f"_forecast_local_{hours_lag}h",
            )

        df_features = df_features.with_columns(
            (
                pl.col(f"temperature_forecast_local_0h")
                / (pl.col(f"temperature_forecast_local_168h") + 1e-3)
            ).alias(f"temperature_forecast_local_0h/168h"),
            (
                pl.col(f"surface_solar_radiation_downwards_forecast_local_0h")
                / (pl.col(f"surface_solar_radiation_downwards_forecast_local_168h") + 1e-3)
            ).alias(f"surface_solar_radiation_downwards_forecast_local_0h/168h"),
        )

        return df_features

    def _add_historical_weather_features(self, df_features):
        df_historical_weather = self.data_storage.df_historical_weather
        df_weather_station_to_county_mapping = (
            self.data_storage.df_weather_station_to_county_mapping
        )

        df_historical_weather = (
            df_historical_weather.with_columns(
                pl.col("latitude").cast(pl.datatypes.Float32),
                pl.col("longitude").cast(pl.datatypes.Float32),
            )
            .join(
                df_weather_station_to_county_mapping,
                how="left",
                on=["longitude", "latitude"],
            )
            .drop("longitude", "latitude")
        )

        df_historical_weather_date = (
            df_historical_weather.group_by("datetime").mean().drop("county")
        )

        df_historical_weather_local = (
            df_historical_weather.filter(pl.col("county").is_not_null())
            .group_by("county", "datetime")
            .mean()
        )

        for hours_lag in [2 * 24, 7 * 24]:
            df_features = df_features.join(
                df_historical_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on="datetime",
                how="left",
                suffix=f"_historical_{hours_lag}h",
            )
            df_features = df_features.join(
                df_historical_weather_local.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on=["county", "datetime"],
                how="left",
                suffix=f"_historical_local_{hours_lag}h",
            )

        for hours_lag in [1 * 24]:
            df_features = df_features.join(
                df_historical_weather_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag),
                    pl.col("datetime").dt.hour().alias("hour"),
                )
                .filter(pl.col("hour") <= 10)
                .drop("hour"),
                on="datetime",
                how="left",
                suffix=f"_historical_{hours_lag}h",
            )

        df_features = df_features.with_columns(
            (
                pl.col(f"temperature_historical_local_48h")
                / (pl.col(f"temperature_historical_local_168h") + 1e-3)
            ).alias(f"temperature_historical_local_48h/168h"),
            (
                pl.col(f"direct_solar_radiation_historical_local_48h")
                / (pl.col(f"direct_solar_radiation_historical_local_168h") + 1e-3)
            ).alias(f"direct_solar_radiation_historical_local_48h/168h"),
            (
                pl.col(f"temperature_historical_24h")
                / (pl.col(f"temperature") + 1e-3)
            ).alias(f"temperature_historical_24h/48h"),
            (
                pl.col(f"direct_solar_radiation_historical_24h")
                / (pl.col(f"direct_solar_radiation") + 1e-3)
            ).alias(f"direct_solar_radiation_historical_24h/48h"),
        )

        return df_features

    def _add_target_features(self, df_features):
        df_target = self.data_storage.df_target

        df_target_all_type_sum = (
            df_target.group_by(["datetime", "county", "is_business", "is_consumption"])
            .sum()
            .drop("product_type")
        )

        df_target_all_county_type_sum = (
            df_target.group_by(["datetime", "is_business", "is_consumption"])
            .sum()
            .drop("product_type", "county")
        )

        for hours_lag in [
            2 * 24,
            3 * 24,
            4 * 24,
            5 * 24,
            6 * 24,
            7 * 24,
            8 * 24,
            9 * 24,
            10 * 24,
            11 * 24,
            12 * 24,
            13 * 24,
            14 * 24,
        ]:
            df_features = df_features.join(
                df_target.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ).rename({"target": f"target_{hours_lag}h"}),
                on=[
                    "county",
                    "is_business",
                    "product_type",
                    "is_consumption",
                    "datetime",
                ],
                how="left",
            )

        for hours_lag in [2 * 24, 3 * 24, 7 * 24, 14 * 24]:
            df_features = df_features.join(
                df_target_all_type_sum.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ).rename({"target": f"target_all_type_sum_{hours_lag}h"}),
                on=["county", "is_business", "is_consumption", "datetime"],
                how="left",
            )

            df_features = df_features.join(
                df_target_all_county_type_sum.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ).rename({"target": f"target_all_county_type_sum_{hours_lag}h"}),
                on=["is_business", "is_consumption", "datetime"],
                how="left",
                suffix=f"_all_county_type_sum_{hours_lag}h",
            )

        cols_for_stats = [
            f"target_{hours_lag}h" for hours_lag in [2 * 24, 3 * 24, 4 * 24, 5 * 24]
        ]
        df_features = df_features.with_columns(
            df_features.select(cols_for_stats).mean(axis=1).alias(f"target_mean"),
            df_features.select(cols_for_stats)
            .transpose()
            .std()
            .transpose()
            .to_series()
            .alias(f"target_std"),
        )

        for target_prefix, lag_nominator, lag_denomonator in [
            ("target", 24 * 7, 24 * 14),
            ("target", 24 * 2, 24 * 9),
            ("target", 24 * 3, 24 * 10),
            ("target", 24 * 2, 24 * 3),
            ("target_all_type_sum", 24 * 2, 24 * 3),
            ("target_all_type_sum", 24 * 7, 24 * 14),
            ("target_all_county_type_sum", 24 * 2, 24 * 3),
            ("target_all_county_type_sum", 24 * 7, 24 * 14),
        ]:
            df_features = df_features.with_columns(
                (
                    pl.col(f"{target_prefix}_{lag_nominator}h")
                    / (pl.col(f"{target_prefix}_{lag_denomonator}h") + 1e-3)
                ).alias(f"{target_prefix}_ratio_{lag_nominator}_{lag_denomonator}")
            )

        return df_features

    def _reduce_memory_usage(self, df_features):
        df_features = df_features.with_columns(pl.col(pl.Float64).cast(pl.Float32))
        return df_features

    def _drop_columns(self, df_features):
        df_features = df_features.drop(
            "date", "datetime", "hour", "dayofyear"
        )
        return df_features

    def _to_pandas(self, df_features, y):
        cat_cols = [
            "county",
            "is_business",
            "product_type",
            "is_consumption",
            "segment",
        ]

        if y is not None:
            df_features = pd.concat([df_features.to_pandas(), y.to_pandas()], axis=1)
        else:
            df_features = df_features.to_pandas()

        df_features[cat_cols] = df_features[cat_cols].astype("category")

        if 'row_id' in df_features.columns:
            df_features = df_features.drop("row_id", axis=1)

        return df_features

    def generate_features(self, df_prediction_items):
        if "target" in df_prediction_items.columns:
            df_prediction_items, y = (
                df_prediction_items.drop("target"),
                df_prediction_items.select("target"),
            )
        else:
            y = None

        df_features = df_prediction_items.with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )

        for add_features in [
            self._add_general_features,
            self._add_client_features,
            self._add_forecast_weather_features,
            self._add_historical_weather_features,
            self._add_target_features,
            self._add_holidays_features,
            self._reduce_memory_usage,
            self._drop_columns,
        ]:
            df_features = add_features(df_features)

        df_features = self._to_pandas(df_features, y)

        return df_features

# Initialisation

In [ ]:
data_storage = DataStorage()
features_generator = FeaturesGenerator(data_storage=data_storage)

# Feature Generation

In [ ]:
df_train_features = features_generator.generate_features(data_storage.df_data)
df_train_features = df_train_features[df_train_features['target'].notnull()]
df_train_features.shape

(1604324, 172)

# LightBGM Models Hyperparameter Optimization

In [ ]:
def lgbm_model(mask,trial,is_diff):
    X = df_train_features[mask].drop(columns=["target"])
    Y = df_train_features[mask]["target"] - (df_train_features[mask]["target_48h"].fillna(0) * is_diff)
    train_size=0.85
    X_train, X_test = X.head(int(len(Y)*train_size)) , X.tail(int(len(Y)*(1-train_size)))
    y_train, y_test = Y.head(int(len(Y)*train_size)) , Y.tail(int(len(Y)*(1-train_size)))
    param = {
        "device": "gpu" ,
        "random_state": 49,
        "objective": "regression_l1",
        "verbose": -1,


        "n_estimators": trial.suggest_int("n_estimators", 2000,10000,step=500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 1),
        "num_leaves": trial.suggest_int("num_leaves", 2**2, 2**8, step=4),
        "max_depth": trial.suggest_int("max_depth", 3, 15,step=1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 1000),
        "lambda_l1": trial.suggest_float("lambda_l1", 0, 5),
        "lambda_l2": trial.suggest_float("lambda_l2", 0, 5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "feature_fraction":trial.suggest_float("feature_fraction", 0.5, 1),
        "feature_fraction_bynode":trial.suggest_float("feature_fraction_bynode", 0.5, 1),
    }
    lgb=LGBMRegressor(**param)
    lgb.fit(X_train, y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="l1",
            )
    pred_lgb=lgb.predict(X_test)
    mae = mean_absolute_error(y_test, pred_lgb)
    return mae

In [ ]:
def Hyperparameter_study(mask,study_name,is_diff,n_trials):
    study=optuna.create_study(direction='minimize',study_name=study_name)
    func = lambda trial: lgbm_model(mask,trial,is_diff)
    study.optimize(func, n_trials=n_trials)
    return study

In [ ]:
def Hyperparameter_study_visualization(study,target_name,study_name):
    optuna.visualization.plot_optimization_history(study,target_name=target_name).show()
    # plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
    optuna.visualization.plot_parallel_coordinate(study,target_name=target_name).show()
    # plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search went and which parts of the space were explored more.
    optuna.visualization.plot_slice(study,target_name=target_name).show()
    #Visualize parameter importances.
    optuna.visualization.plot_param_importances(study,target_name=target_name).show()
    #Visualize empirical distribution function
    optuna.visualization.plot_edf(study,target_name=target_name).show()
    print(f"\tBest value (MAE): {study.best_value:.5f}")
    print(f"\tBest params {study_name}:")
    for key, value in study.best_params.items():
        print(f"\t\t{key}: {value}")

In [ ]:
consumption = df_train_features["is_consumption"] == 1
production = df_train_features["is_consumption"] == 0
n_trials = 50

## Consumption_model_lgbm

In [ ]:
study_consumption = Hyperparameter_study(mask=consumption,study_name="Consumption_model_lgbm",is_diff=0 , n_trials=n_trials)

[I 2024-01-25 02:35:15,612] A new study created in memory with name: Consumption_model_lgbm
[I 2024-01-25 02:36:13,155] Trial 0 finished with value: 107.02540749701076 and parameters: {'n_estimators': 10000, 'learning_rate': 0.7591710108240668, 'num_leaves': 212, 'max_depth': 12, 'min_data_in_leaf': 159, 'lambda_l1': 2.211118511775287, 'lambda_l2': 1.7886293167995175, 'min_gain_to_split': 2.8929018055983846, 'feature_fraction': 0.506736842012727, 'feature_fraction_bynode': 0.7868139579639506}. Best is trial 0 with value: 107.02540749701076.
[I 2024-01-25 02:39:01,850] Trial 1 finished with value: 59.87141842614229 and parameters: {'n_estimators': 8000, 'learning_rate': 0.028311420868639095, 'num_leaves': 20, 'max_depth': 15, 'min_data_in_leaf': 893, 'lambda_l1': 0.4145124190727417, 'lambda_l2': 0.01731033067193022, 'min_gain_to_split': 3.78528539131029, 'feature_fraction': 0.8541045993630019, 'feature_fraction_bynode': 0.9284479888028461}. Best is trial 1 with value: 59.87141842614229.

In [ ]:
Hyperparameter_study_visualization(study = study_consumption,target_name="MAE",study_name="Consumption_model_lgbm")

	Best value (MAE): 59.32963
	Best params Consumption_model_lgbm:
		n_estimators: 7500
		learning_rate: 0.04592010474307811
		num_leaves: 56
		max_depth: 14
		min_data_in_leaf: 375
		lambda_l1: 0.03410311762087645
		lambda_l2: 0.9845952843721262
		min_gain_to_split: 0.5296254111362524
		feature_fraction: 0.6370684599391575
		feature_fraction_bynode: 0.677857615956361


## Production_model_lgbm

In [ ]:
study_production = Hyperparameter_study(mask=production,study_name="Production_model_lgbm",is_diff=0 , n_trials=n_trials)

[I 2024-01-25 04:01:23,798] A new study created in memory with name: Production_model_lgbm
[I 2024-01-25 04:06:45,570] Trial 0 finished with value: 73.72739656522619 and parameters: {'n_estimators': 8000, 'learning_rate': 0.9914000222158804, 'num_leaves': 152, 'max_depth': 8, 'min_data_in_leaf': 235, 'lambda_l1': 1.720409342517335, 'lambda_l2': 0.8246793561919341, 'min_gain_to_split': 1.8832947390690453, 'feature_fraction': 0.8557853475363513, 'feature_fraction_bynode': 0.8274918296854876}. Best is trial 0 with value: 73.72739656522619.
[I 2024-01-25 04:08:11,346] Trial 1 finished with value: 62.50458514739357 and parameters: {'n_estimators': 3500, 'learning_rate': 0.5033381779067149, 'num_leaves': 224, 'max_depth': 13, 'min_data_in_leaf': 91, 'lambda_l1': 1.1261538182271984, 'lambda_l2': 2.605573150645606, 'min_gain_to_split': 14.300394140894642, 'feature_fraction': 0.5722576463053066, 'feature_fraction_bynode': 0.5113909201271107}. Best is trial 1 with value: 62.50458514739357.
[I 20

In [ ]:
Hyperparameter_study_visualization(study = study_production,target_name="MAE",study_name="Production_model_lgbm")

	Best value (MAE): 53.69970
	Best params Production_model_lgbm:
		n_estimators: 6500
		learning_rate: 0.0807960024938446
		num_leaves: 244
		max_depth: 12
		min_data_in_leaf: 873
		lambda_l1: 2.4284658083286974
		lambda_l2: 2.9442603917437324
		min_gain_to_split: 1.3531656889137775
		feature_fraction: 0.9696603522093273
		feature_fraction_bynode: 0.7591695281520368


## Consumption_48h_diif_model_lgbm

In [ ]:
study_consumption_48h_diif = Hyperparameter_study(mask=consumption, study_name="Consumption_48h_diif_model_lgbm", is_diff=1 , n_trials=n_trials)

[I 2024-01-25 06:08:22,485] A new study created in memory with name: Consumption_48h_diif_model_lgbm
[I 2024-01-25 06:08:50,489] Trial 0 finished with value: 69.96571335475006 and parameters: {'n_estimators': 7500, 'learning_rate': 0.39201529242851096, 'num_leaves': 44, 'max_depth': 14, 'min_data_in_leaf': 78, 'lambda_l1': 0.32409469955704373, 'lambda_l2': 0.5492717121219121, 'min_gain_to_split': 13.11525386163472, 'feature_fraction': 0.8030663925574923, 'feature_fraction_bynode': 0.7139062387460928}. Best is trial 0 with value: 69.96571335475006.
[I 2024-01-25 06:09:17,448] Trial 1 finished with value: 81.49527911528106 and parameters: {'n_estimators': 7000, 'learning_rate': 0.7808669373117687, 'num_leaves': 220, 'max_depth': 14, 'min_data_in_leaf': 395, 'lambda_l1': 1.7349869569097798, 'lambda_l2': 0.5936812900335797, 'min_gain_to_split': 6.146951433909513, 'feature_fraction': 0.8922568575843848, 'feature_fraction_bynode': 0.5715589900669429}. Best is trial 0 with value: 69.965713354

In [ ]:
Hyperparameter_study_visualization(study = study_consumption_48h_diif,target_name="MAE",study_name="Consumption_48h_diif_model_lgbm")

	Best value (MAE): 58.90186
	Best params Consumption_48h_diif_model_lgbm:
		n_estimators: 2500
		learning_rate: 0.02993382083458325
		num_leaves: 256
		max_depth: 9
		min_data_in_leaf: 379
		lambda_l1: 4.304115523255496
		lambda_l2: 0.6953892324335332
		min_gain_to_split: 1.0845923138763394
		feature_fraction: 0.7859452310000432
		feature_fraction_bynode: 0.5603115720404521


## Production_48h_diif_model_lgbm

In [ ]:
study_production_48h_diif = Hyperparameter_study(mask=production, study_name="Production_48h_diif_model_lgbm", is_diff=1 , n_trials=n_trials)

[I 2024-01-25 07:33:03,666] A new study created in memory with name: Production_48h_diif_model_lgbm
[I 2024-01-25 07:34:31,348] Trial 0 finished with value: 55.9200200002262 and parameters: {'n_estimators': 7500, 'learning_rate': 0.37817732441463, 'num_leaves': 144, 'max_depth': 12, 'min_data_in_leaf': 40, 'lambda_l1': 2.682872801974563, 'lambda_l2': 2.613300984948903, 'min_gain_to_split': 8.452788165299685, 'feature_fraction': 0.6844296386623636, 'feature_fraction_bynode': 0.6197191561998378}. Best is trial 0 with value: 55.9200200002262.
[I 2024-01-25 07:35:25,309] Trial 1 finished with value: 58.3047669838195 and parameters: {'n_estimators': 4500, 'learning_rate': 0.9031896920007343, 'num_leaves': 8, 'max_depth': 8, 'min_data_in_leaf': 758, 'lambda_l1': 2.5677366254279788, 'lambda_l2': 0.8584732633545072, 'min_gain_to_split': 6.498269742749611, 'feature_fraction': 0.6274215756657233, 'feature_fraction_bynode': 0.5897057550513923}. Best is trial 0 with value: 55.9200200002262.
[I 202

In [ ]:
Hyperparameter_study_visualization(study = study_consumption_48h_diif,target_name="MAE",study_name="Production_48h_diif_model_lgbm")

	Best value (MAE): 58.90186
	Best params Production_48h_diif_model_lgbm:
		n_estimators: 2500
		learning_rate: 0.02993382083458325
		num_leaves: 256
		max_depth: 9
		min_data_in_leaf: 379
		lambda_l1: 4.304115523255496
		lambda_l2: 0.6953892324335332
		min_gain_to_split: 1.0845923138763394
		feature_fraction: 0.7859452310000432
		feature_fraction_bynode: 0.5603115720404521
